# 2023 IEEE SciVis Contest
### Neuronal Network Simulations of the Human Brain

Authors: Seda den Boer, Dominique Weltevreden

Course: Scientific Visualisation and Virtual Reality, UvA

Semester: Fall 2023

#### Research question:
How does setting individual calcium targets for neurons influence the general brain activity and connectivity?

### Requirements:
Required packages and correct versions can be downloaded with `pip install -r requirements.txt`

#### Prerequisites:
* Create an empty `data` filemap.
* Download the 2023 IEEE SciVis Contest `viz-no-network` and `viz-calcium` data from the following page https://rwth-aachen.sciebo.de/s/KNTo1vgT0JZyGJx, and store it in `data` filemap.
* Unzip the monitors files.
* Make sure to create two new empty folders `monitors_extracted` and `files` in each simulation filemap.

Below the necessary data (filemaps) are shown:
```
└───data
    ├───viz-calcium
    │   ├───files
    │   ├───monitors
    │   │   • 0_{neuron_id}.csv
    │   ├───monitors_extracted
    │   └───positions
    │       • rank_0_positions.txt
    │   • calcium_targets.txt
    └───viz-no-network
        ├───files
        ├───monitors
        │   • 0_{neuron_id}.csv
        ├───monitors_extracted
        └───positions
            • rank_0_positions.txt
```

In [1]:
import numpy as np
import pandas as pd
from typing import List
import os

## Step 1: extracting relevant variable data from monitors files

In [2]:
def get_data(simulation_type: str, column_name: str) -> pd.DataFrame:
    """
    Get the data from the csv files and save it to a pickle file.

    Args:
        simulation_type (str): The type of simulation.
        column_name (str): The name of the column to extract.
    
    Returns:
        pd.DataFrame: The dataframe with the extracted data.
    """
    if simulation_type == 'no-network':
        file_directory = 'viz-no-network'
    elif simulation_type == 'calcium':
        file_directory = 'viz-calcium'

    gathered = []
    df_combined = pd.DataFrame()
    
    # Gather all the calcium levels
    for i in range(50000):
        # Get file path
        monitor_file = f'data/{file_directory}/monitors/0_{i}.csv'
        # Put it in a dataframe
        df = pd.read_csv(monitor_file, header=None, sep=';',
                         names=['step', f'fired_{i}', f'fired_fraction_{i}', f'x_{i}', f'secondary_variable_{i}',
                                f'calcium_{i}', f'target_calcium_{i}', f'synaptic_input_{i}', f'background_activity_{i}',
                                f'grown_axons_{i}', f'connected_axons_{i}', f'grown_excitatory_dendrites_{i}', f'connected_excitatory_dendrites_{i}'])

        # Set step column as [0, 100, 200, 300, ..., 1000000]
        df['step'] = np.arange(0, 1000000, 100)

        # Copy the last row to the end of the dataframe and increment the index
        df.loc[len(df)] = df.iloc[-1]

        # Set the step of the last row to 1000000
        df['step'].iloc[-1] = 1000000

        # Only keep the rows with step [0, 10000, 20000, ..., 1000000]
        df = df[df['step'] % 10000 == 0]

        # Add calcium levels to a list
        gathered.append(df[f'{column_name}_{i}'])

    # Combine all the calcium levels into a dataframe
    df_combined = pd.concat(gathered, axis=1)

    # Add steps as first column
    steps = np.arange(0, 1010000, 10000)
    df_combined.insert(0, 'step', steps)

    # Set step as the index column
    df_combined.set_index('step', inplace=True)

    # Create the directory if it doesn't exist
    output_dir = f'data/{file_directory}/monitors_extracted'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Save the dataframe to a csv file
    output_file = f'{output_dir}/{simulation_type}_{column_name}.pkl'
    
    # Save as pickle file
    df_combined.to_pickle(output_file)

    return df_combined

In [3]:
# This piece of code takes a while to run, uncomment only if you want to run it
"""
columns = ['calcium', 'connected_axons', 'fired', 'x']

for column in columns:
    get_data('no-network', column)
    get_data('calcium', column)
"""

"\ncolumns = ['calcium', 'connected_axons', 'fired', 'x']\n\nfor column in columns:\n    get_data('no-network', column)\n    get_data('calcium', column)\n"

## Step 2: merging extracted data from monitors and positional data into a combined .csv file

In [4]:
# Indicate dataset to process ('calcium', 'no-network')
sim = 'calcium'

In [5]:
# Get the position file, convert to dataframe and clean up
position_file_loc = f'data/viz-{sim}/positions/rank_0_positions.txt'
df_positions = pd.read_csv(position_file_loc, delimiter=r'\s+', skiprows=8, names=['localid', 'posx', 'posy', 'posz', 'area', 'type'])
df_positions['area'] = df_positions['area'].str.replace('area_', '')
df_positions.drop('type', axis=1, inplace=True)
display(df_positions)

,localid,posx,posy,posz,area
0,1,88.017654,143.889110,83.259131,43
1,2,88.071787,143.792200,83.237149,43
2,3,88.093434,143.951900,83.168331,43
3,4,88.063898,143.819810,83.317563,43
4,5,88.066131,143.973540,83.238470,43
...,...,...,...,...,...
49995,49996,86.890937,1.931412,79.944581,3
49996,49997,86.764390,1.834104,80.037049,3
49997,49998,86.699561,1.970323,79.967185,3
49998,49999,86.841367,1.944717,80.029320,3


In [6]:
calcium_file_loc = f'data/viz-{sim}/monitors_extracted/{sim}_calcium.pkl'

# Read the calcium file and transpose it
calcium = pd.read_pickle(calcium_file_loc).T

In [7]:
connected_axons_file_loc = f'data/viz-{sim}/monitors_extracted/{sim}_connected_axons.pkl'

# Read the connected axons file and transpose it
connected_axons = pd.read_pickle(connected_axons_file_loc).T

In [8]:
fired_file_loc = f'data/viz-{sim}/monitors_extracted/{sim}_fired.pkl'

# Read the fired file and transpose it
fired = pd.read_pickle(fired_file_loc).T

In [9]:
x_file_loc = f'data/viz-{sim}/monitors_extracted/{sim}_x.pkl'

# Read the x file and transpose it
x = pd.read_pickle(x_file_loc).T

In [10]:
if sim == 'calcium':
    # Read calcium targets
    calcium_targets_df = pd.read_csv(f'data/viz-{sim}/calcium_targets.txt', sep="\t", names=["neuron_id", "initial_calcium", "target_calcium"])
    calcium_targets_df = calcium_targets_df.iloc[3:]

    # Correct neuron ids
    calcium_targets_df['neuron_id'] = calcium_targets_df['neuron_id'].astype(int)
    calcium_targets_df['neuron_id'] = calcium_targets_df['neuron_id'] - 1
    calcium_targets_df.set_index('neuron_id', inplace=True)
    display(calcium_targets_df)

    # Set the target calcium as a numpy array ant make the values floats
    calcium_targets = calcium_targets_df['target_calcium'].to_numpy().astype(float)
    print(calcium_targets)
else:
    # Set the calcium target to 0.7 for all neurons
    calcium_targets = np.full(50000, 0.7)


,initial_calcium,target_calcium
neuron_id,,
0,0.0,0.669532088
1,0.0,0.68687669
2,0.0,0.64210661
3,0.0,0.757728899
4,0.0,0.664595307
...,...,...
49995,0.0,0.620906382
49996,0.0,0.688855601
49997,0.0,0.603795437


[0.66953209 0.68687669 0.64210661 ... 0.60379544 0.65278311 0.72715949]


In [11]:
def process_data(sim: str, scalar_data: List[pd.DataFrame], columns: List[str], calcium_targets: np.array, savefiles: bool = False) -> List[pd.DataFrame]:
    """Generates a list of dataframes for each timestep of 
    the visualisation. The dataframe contains the positions
    of the neurons, area, and correspondong scalar values that
    have been retrieved from the monitors files.

    Args:
        sim (str): simulation type
        scalar_data (List[pd.DataFrame]): list with scalar value dataframes
        columns (List[str]): names of the scalar data
        calcium_targets (np.array): array with calcium targets
        savefiles (bool): whether to save the vtk files

    Returns:
        List[pd.Dataframe]: list of dataframes with complete data per timestep
    """
    df_list = []
    timesteps = np.arange(0, 1010000, 10000)

    for timestep in timesteps:
        df = df_positions.copy()

        # Add scalar values to the dataframe
        for i, scalar_df in enumerate(scalar_data):
            df[columns[i]] = scalar_df[timestep].reset_index(drop=True)

        # Add calcium targets to the dataframe
        df['target_calcium'] = calcium_targets

        # Add a column with the net calcium level
        df['calcium_difference'] = df['target_calcium'] - df['calcium']

        df_list.append(df)
        
        # Save the point cloud as a VTK file
        if savefiles:
            csv_filename = f'data/viz-{sim}/files/positions_scalars_{timestep}.csv'
            df.to_csv(csv_filename, index=False)

    return df_list

In [12]:
# Process the data
df_list = process_data(
                    sim=sim,
                    scalar_data=[calcium, connected_axons, fired, x],
                    columns=['calcium', 'connected_axons', 'fired', 'x'],
                    calcium_targets=calcium_targets,
                    savefiles=False
                )

In [13]:
# Check if everything went well
display(df_list[0])

,localid,posx,posy,posz,area,calcium,connected_axons,fired,x,target_calcium,calcium_difference
0,1,88.017654,143.889110,83.259131,43,0.003982,11.0,0.0,-60.9230,0.669532,0.665550
1,2,88.071787,143.792200,83.237149,43,0.003983,14.0,0.0,-64.2071,0.686877,0.682893
2,3,88.093434,143.951900,83.168331,43,0.003983,20.0,0.0,-64.0445,0.642107,0.638124
3,4,88.063898,143.819810,83.317563,43,0.003983,12.0,0.0,-64.1507,0.757729,0.753746
4,5,88.066131,143.973540,83.238470,43,0.005973,18.0,0.0,-63.1058,0.664595,0.658623
...,...,...,...,...,...,...,...,...,...,...,...
49995,49996,86.890937,1.931412,79.944581,3,0.005978,16.0,0.0,-61.1174,0.620906,0.614929
49996,49997,86.764390,1.834104,80.037049,3,0.003983,15.0,0.0,-63.8706,0.688856,0.684872
49997,49998,86.699561,1.970323,79.967185,3,0.005978,16.0,0.0,-60.7481,0.603795,0.597818
49998,49999,86.841367,1.944717,80.029320,3,0.005972,14.0,0.0,-61.5825,0.652783,0.646811
